In [33]:
import json
import re

import pandas as pd
import requests

import wmfdata as wmf
from wmfdata.utils import get_dblist

In [34]:
# Use all.dblist instead of a sites table, which could include deleted wikis
wikis = get_dblist("all")

# Make it a dataframe with the proper column name so it can be merged 
wikis = pd.DataFrame(wikis, columns=["database_code"])

In [35]:
# Join to additional data from sites table, excluding entries not found in all.dblist
sites = wmf.mariadb.run("""
SELECT
    site_global_key AS database_code,
    CONCAT(TRIM(LEADING "." FROM REVERSE(site_domain))) AS domain_name,
    site_group AS database_group,
    site_language AS language_code
FROM sites
""", "metawiki")

wikis = wikis.merge(sites, how="left", on="database_code", sort="True").set_index("database_code")

In [36]:
# Check for wikis with null data (e.g. newly created wikis not yet added to the sites table)
wikis[wikis.isnull().any(axis=1)]

,domain_name,database_group,language_code
database_code,,,


In [37]:
# Add missing data. Remove manual additions once no longer needed
extra_wikis = pd.DataFrame([
  # Example: ("gcrwiki", "gcr.wikipedia.org", "wikipedia", "gcr"),
], columns=["database_code", "domain_name", "database_group", "language_code"]
).set_index("database_code")

wikis.update(extra_wikis, overwrite=False)

# Language names

In [48]:
lang_urls = [
    "https://raw.githubusercontent.com/wikimedia/mediawiki-extensions-cldr/master/CldrNames/CldrNamesEn.php",
    "https://raw.githubusercontent.com/wikimedia/mediawiki-extensions-cldr/master/LocalNames/LocalNamesEn.php"
]

def get_lang_names(url):
    r = requests.get(url)
    m = re.search(r"languageNames = (\[[\s\S]+?\])", r.text)
    php_ln = m.group(1)
    
    json_ln = php_ln
    repl = [
        # Convert from PHP array format to JSON
        (" =>", ":"),
        ("\[", "{"),
        ("\]", "}"),
        # Trailing commas will cause problems
        (",\n}", "\n}"),
        # ...so will single quotes
        ("'", '"'),
        # ...and comments
        (r"/\*[\s\S]*?\*/", ""),
        (r"#(.*?)\n", ""),
        # One hack to deal with a single quote in a language name
        ('O"odham', "O'odham")
    ]
    for old, new in repl:
        json_ln = re.sub(old, new, json_ln)
    
    py_ln = json.loads(json_ln)
    return py_ln

langs = {}
for url in lang_urls:
    langs.update(get_lang_names(url))

wikis["language_name"] = wikis["language_code"].apply(langs.get)

In [52]:
# Check for wikis with null language names since some are not included in CLDR
null_lang_wikis = wikis[wikis["language_name"].isna()].copy()
set(null_lang_wikis["language_code"])

{'als',
 'atj',
 'diq',
 'fiu-vro',
 'map-bms',
 'nah',
 'pih',
 'simple',
 'szy',
 'tay'}

In [56]:
# Add missing language names. Remove manual additions once no longer needed. 
extra_langs = {
    "als": "Alsatian",
    "atj": "Atikamekw",
    "diq": "Zazaki",
    "fiu-vro": "Võro",
    "map-bms": "Banyumasan",
    "nah": "Nahuatl",
    "pih": "Norfuk-Pitkern",
    "simple": "Simple English",
    "szy": "Sakizaya",
    "tay": "Atayal"
}

null_lang_wikis["language_name"] = null_lang_wikis["language_code"].apply(extra_langs.get)
wikis.update(null_lang_wikis, overwrite=False)

# Status and access

In [73]:
closed = get_dblist("closed")
private = get_dblist("private")
fishbowl = get_dblist("fishbowl")
editability_private = private.copy()
editability_private.extend(fishbowl)

def apply_to_index(df, true_list, true_label, false_label):
    idx_ser = df.index.to_series()
    return idx_ser.isin(true_list).apply(lambda x: true_label if x else false_label)

wikis = (
    wikis.assign(
        status=lambda df: apply_to_index(df, closed, "closed", "open"),
        visibility=lambda df: apply_to_index(df, private, "private", "public"),
        editability=lambda df: apply_to_index(df, editability_private, "private", "public")
    )
)

# Site names

In [74]:
project_names = requests.get(
    "https://raw.githubusercontent.com/wikimedia/mediawiki-extensions-WikimediaMessages/master/" +
    "i18n/wikimediaprojectnames/en.json"
).text

replacements = [
    (r'"project-localized-name-(\w*)":', r'"\1":'),
    (r'"@metadata": \[.*\],', '')
]

for old, new in replacements:
    project_names = re.sub(old, new, project_names)

project_names = json.loads(project_names)
project_names = pd.DataFrame.from_dict(
    project_names, 
    orient="index", 
    columns=["english_name"]
).rename_axis("database_code")

In [75]:
wikis = wikis.merge(project_names, on="database_code")

# Write CSV

In [77]:
wikis.to_csv("../wikis.csv")